In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn. metrics import mean_squared_log_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.linear_model import Lasso, Ridge

In [2]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [3]:
train.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [4]:
train['date_time'] = pd.to_datetime(train['date_time'])
test['date_time'] = pd.to_datetime(test['date_time'])

In [5]:
print(f"The starting date in train data in {train['date_time'].min()}")
print(f"The ending date in train data in {train['date_time'].max()}")

The starting date in train data in 2010-03-10 18:00:00
The ending date in train data in 2011-01-01 00:00:00


In [6]:
print(f"The starting date in test data in {test['date_time'].min()}")
print(f"The ending date in test data in {test['date_time'].max()}")

The starting date in test data in 2011-01-01 00:00:00
The ending date in test data in 2011-04-04 14:00:00


In [7]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [8]:
# Features to use:
columns_to_use = ['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']
y_train = train[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
x_train = train.drop(['date_time','target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis=1)
x_test = test.drop(['date_time'], axis=1)
x_train.shape, x_test.shape, y_train.shape

((7111, 8), (2247, 8), (7111, 3))

In [9]:
y_train.iloc[:, 2]

0       167.7
1        98.9
2       127.1
3       177.2
4       121.8
        ...  
7106    191.1
7107    221.3
7108    227.4
7109    199.8
7110    186.5
Name: target_nitrogen_oxides, Length: 7111, dtype: float64

In [10]:
xgb = SVR()
xgb.fit(x_train, y_train.iloc[:, 0])
submission['target_carbon_monoxide'] = xgb.predict(x_test)

xgb = SVR()
xgb.fit(x_train, y_train.iloc[:, 1])
submission['target_benzene'] = xgb.predict(x_test)

xgb = SVR()
xgb.fit(x_train, y_train.iloc[:, 2])
submission['target_nitrogen_oxides'] = xgb.predict(x_test)

submission.head()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,1.915729,5.991015,263.393974
1,2011-01-01 01:00:00,2.627029,9.001099,345.849897
2,2011-01-01 02:00:00,2.306136,8.346452,288.529597
3,2011-01-01 03:00:00,2.392457,8.909213,281.456846
4,2011-01-01 04:00:00,1.866613,6.858150,230.660041


In [11]:
submission.to_csv('submission.csv', index=False)